<a href="https://colab.research.google.com/github/yourpandaboy/ViV/blob/main/ViV_tester.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


In [3]:
df = pd.read_csv("/content/drive/MyDrive/okcupid/okcupid_profiles.csv")

In [4]:
bios = df[["essay0"]].dropna() #get essay0 and dropna
test = bios.loc[50000:] #bios for unseen bios

In [5]:
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [7]:
#simple cleaning
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

#data_words = list(sent_to_words(data))

In [8]:
#lemmatization
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [9]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [10]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [11]:
# load pickle
import pickle
model = pickle.load(open("drive/My Drive/optimal_model_45score.pkl", "rb"))

In [12]:
model_predictor = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(model, gamma_threshold=0.001, iterations=50)

In [14]:
# testing unseen data and see which topic it belongs
test[:1].values

array([["lets see here i have a lot of things going on in my life so i guess there is just too much to really say.  i'm just a fun loving person who works and works and studies and works a lot so yea i do need a little bit of time to have fun with new people once and a while and all lol"]],
      dtype=object)

In [15]:
text=pd.DataFrame({'text':["lets see here i have a lot of things going on in my life so i guess there is just too much to really say.  i'm just a fun loving person who works and works and studies and works a lot so yea i do need a little bit of time to have fun with new people once and a while and all lol"]})


data_words = list(sent_to_words(text['text']))
data_nostop= remove_stopwords(data_words)
data_lemma = lemmatization(data_nostop, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
id2word = corpora.Dictionary(data_lemma)
corpus = [id2word.doc2bow(text) for text in data_lemma]


#  predicting new text which is in text dataframe  
new_text_corpus =  id2word.doc2bow(text['text'][0].split())

In [16]:
model_predictor[new_text_corpus] #highest score at topic 14 and 2, please refer to the word cloud to see the topics better.

[(0, 0.05586481872554492),
 (1, 0.08359820348473622),
 (2, 0.05485567001225815),
 (3, 0.06724632874431852),
 (4, 0.053441727204084924),
 (5, 0.07128743842931094),
 (6, 0.052276209392113436),
 (7, 0.06168860165900378),
 (8, 0.04893378514302692),
 (9, 0.05820906617825845),
 (10, 0.05526598526139909),
 (11, 0.07320271114371739),
 (12, 0.05389315233918146),
 (13, 0.10490295426095443),
 (14, 0.04807692307692308),
 (15, 0.05725642494516829)]

In [18]:
# test another unseen data
test[3:4].values

array([['from my facebook profile:  well about me ...i like to take pictures, go to the movies (sci-fi / comedy / adventure) , hang out with my friends, go dancing, listen to music, or just grab a good book and relax ... i have some pics on flickr, username is nutation  i am nerdy, interesting, and stable']],
      dtype=object)

In [19]:
text=pd.DataFrame({'text':['from my facebook profile:  well about me ...i like to take pictures, go to the movies (sci-fi / comedy / adventure) , hang out with my friends, go dancing, listen to music, or just grab a good book and relax ... i have some pics on flickr, username is nutation  i am nerdy, interesting, and stable']})


data_words = list(sent_to_words(text['text']))
lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
id2word = corpora.Dictionary(data_words)
corpus = [id2word.doc2bow(text) for text in data_words]


#  predicting new text which is in text dataframe  
new_text_corpus =  id2word.doc2bow(text['text'][0].split())

In [20]:
model_predictor[new_text_corpus] #topics 14 and 4

[(0, 0.055760991589720824),
 (1, 0.06540447743853307),
 (2, 0.046519911518110614),
 (3, 0.11941354644819976),
 (4, 0.038842228816230756),
 (5, 0.08682491154567427),
 (6, 0.04278552305588437),
 (7, 0.057287609977288566),
 (8, 0.04596000241778545),
 (9, 0.06351741566409252),
 (10, 0.037654795391328245),
 (11, 0.06993557260321176),
 (12, 0.05987039625663064),
 (13, 0.10710953189391917),
 (14, 0.04429835183602291),
 (15, 0.05881473354736708)]

In [21]:
#Questions for Alec:
#1. Am i doing it the preprocessing of the unseen data correctly? 
#2. Do you think that "model_predictor[new_text_corpus]" could work on a pipeline? THANK YOU SO MUCH!